In [1]:
import os
import warnings
import sys
import datetime

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import pickle

In [2]:
sys.path.append(os.path.abspath(os.path.join('E:\\10xAccademy_Practice\\Week 3\\Week 3 Challenge\\Sales Data Prediction\\Data_preparation')))
"""
Import required Data processing scripts
"""

from Data_Loader import Data_Loader 
from Preprocess_Working_Data import Data_Preprocessing
from Make_Plots import Make_Plots

In [3]:
DL=Data_Loader()
dp=Data_Preprocessing()
mp=Make_Plots()

In [4]:
train_data=DL.read_csv("E:\\10xAccademy_Practice\\Week 3\\Week 3 Challenge\\Sales Data Prediction\\Data\\rossmann-store-sales\\Train_Data_AfterEDA.csv")
test_data=DL.read_csv("E:\\10xAccademy_Practice\\Week 3\\Week 3 Challenge\\Sales Data Prediction\\Data\\rossmann-store-sales\\Test_Data_AfterEDA.csv ")

In [5]:
train_data.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,CompetitionDistance,...,Promo2SinceYear,Year,Month,Day,weekday,weekofyear,weekend,PromoInterval,Assortment,StoreType
0,2013-01-01,1,1,0,0,0,0,a,1,1270.0,...,2012.0,2013,1,1,1,1,0,1,0,2
1,2013-01-02,1,2,5530,668,1,0,0,1,1270.0,...,2012.0,2013,1,2,2,1,0,1,0,2
2,2013-01-03,1,3,4327,578,1,0,0,1,1270.0,...,2012.0,2013,1,3,3,1,0,1,0,2
3,2013-01-04,1,4,4486,619,1,0,0,1,1270.0,...,2012.0,2013,1,4,4,1,0,1,0,2
4,2013-01-05,1,5,4997,635,1,0,0,1,1270.0,...,2012.0,2013,1,5,5,1,1,1,0,2


In [6]:
test_data.head()

,Date,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,weekday,weekofyear,weekend
0,2015-08-01,40233,1,5,1.0,0,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,1,5,31,1
1,2015-08-02,39377,1,6,0.0,0,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,2,6,31,1
2,2015-08-03,38521,1,0,1.0,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,3,0,32,1
3,2015-08-04,37665,1,1,1.0,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,4,1,32,0
4,2015-08-05,36809,1,2,1.0,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,5,2,32,0


In [8]:
train_data.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,CompetitionDistance,...,Promo2SinceYear,Year,Month,Day,weekday,weekofyear,weekend,PromoInterval,Assortment,StoreType
0,2013-01-01,1,1,0,0,0,0,a,1,1270.0,...,2012.0,2013,1,1,1,1,0,1,0,2
1,2013-01-02,1,2,5530,668,1,0,0,1,1270.0,...,2012.0,2013,1,2,2,1,0,1,0,2
2,2013-01-03,1,3,4327,578,1,0,0,1,1270.0,...,2012.0,2013,1,3,3,1,0,1,0,2
3,2013-01-04,1,4,4486,619,1,0,0,1,1270.0,...,2012.0,2013,1,4,4,1,0,1,0,2
4,2013-01-05,1,5,4997,635,1,0,0,1,1270.0,...,2012.0,2013,1,5,5,1,1,1,0,2


In [ ]:
test_data.drop('Unnamed: 0',inplace = True, axis=1)  #Because it is not in the test set

In [9]:
test_data.head()

,Date,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,weekday,weekofyear,weekend
0,2015-08-01,40233,1,5,1.0,0,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,1,5,31,1
1,2015-08-02,39377,1,6,0.0,0,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,2,6,31,1
2,2015-08-03,38521,1,0,1.0,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,3,0,32,1
3,2015-08-04,37665,1,1,1.0,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,4,1,32,0
4,2015-08-05,36809,1,2,1.0,1,0,1,c,a,...,0,22.0,2012.0,"Jan,Apr,Jul,Oct",2015,8,5,2,32,0


In [10]:
train_data.isnull().sum()

Date                         0
Store                        0
DayOfWeek                    0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
Year                         0
Month                        0
Day                          0
weekday                      0
weekofyear                   0
weekend                      0
PromoInterval                0
Assortment                   0
StoreType                    0
dtype: int64

In [11]:
test_data.isnull().sum()

Date                         0
Id                           0
Store                        0
DayOfWeek                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
Year                         0
Month                        0
Day                          0
weekday                      0
weekofyear                   0
weekend                      0
dtype: int64

In [ ]:
#train_data.drop('Unnamed: 0',inplace = True, axis=1)  #Because it is not in the test set

In [12]:
print('Distinct number of Stores:', len(train_data['Store'].unique()))
print('Distinct number of Stores:', len(train_data['Date'].unique()))
print('Average daily sales of all Stores:', round(train_data['Sales'].mean(), 2))

Distinct number of Stores: 1115
Distinct number of Stores: 942
Average daily sales of all Stores: 5773.82


In [13]:
train_data.drop(['StateHoliday'], axis=1, inplace=True)

In [14]:
train_data.set_index('Date', inplace=True)

In [15]:
train_data.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,...,Promo2SinceYear,Year,Month,Day,weekday,weekofyear,weekend,PromoInterval,Assortment,StoreType
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1,1,0,0,0,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,1,1,1,0,1,0,2
2013-01-02,1,2,5530,668,1,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,2,2,1,0,1,0,2
2013-01-03,1,3,4327,578,1,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,3,3,1,0,1,0,2
2013-01-04,1,4,4486,619,1,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,4,4,1,0,1,0,2
2013-01-05,1,5,4997,635,1,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,5,5,1,1,1,0,2


In [16]:
train_data = train_data[train_data['Open'] == 1]
train_data = train_data[train_data['Sales'] > 0.0]

In [17]:
train_data.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,...,Promo2SinceYear,Year,Month,Day,weekday,weekofyear,weekend,PromoInterval,Assortment,StoreType
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,1,2,5530,668,1,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,2,2,1,0,1,0,2
2013-01-03,1,3,4327,578,1,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,3,3,1,0,1,0,2
2013-01-04,1,4,4486,619,1,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,4,4,1,0,1,0,2
2013-01-05,1,5,4997,635,1,0,1,1270.0,9.0,2008.0,...,2012.0,2013,1,5,5,1,1,1,0,2
2013-01-07,1,0,7176,785,1,1,1,1270.0,9.0,2008.0,...,2012.0,2013,1,7,0,2,1,1,0,2


# Split the data

In [18]:
X = train_data.drop('Sales',axis=1)
y = train_data['Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [19]:
data_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])

pipeline = Pipeline(steps = [
               ('preprocessor', data_transformer)
              ,('regressor',RandomForestRegressor(max_depth=64, random_state=0))
           ])

In [20]:
rf_model = pipeline.fit(X_train, y_train)
print(rf_model)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('imputer', SimpleImputer()),
                                 ('scaler', StandardScaler())])),
                ('regressor',
                 RandomForestRegressor(max_depth=64, random_state=0))])


In [21]:
rf_model.score(X_test, y_test)

0.9764074047175012

In [ ]:
# Save model based on time st
rf_model_path = 'E:\\10xAccademy_Practice\\Week 3\\Week 3 Challenge\\Sales Data Prediction\\Models\\' + "RandomForest_Model"+datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.pkl'
pickle.dump(rf_model, open(rf_model_path, 'wb'))